<a href="https://colab.research.google.com/github/leonrafael29/W266_Final_Project/blob/main/mBART/Single_MBart_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install requirements

In [ ]:
!pip install datasets -q
!pip install sentencepiece -q
!pip install transformers -q
!pip install git+https://github.com/google-research/bleurt.git -q

# !wget -N https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip . -q
# !unzip -q -n BLEURT-20.zip


     |████████████████████████████████| 451 kB 34.7 MB/s 
     |████████████████████████████████| 212 kB 91.0 MB/s 
     |████████████████████████████████| 182 kB 95.8 MB/s 
     |████████████████████████████████| 115 kB 90.5 MB/s 
     |████████████████████████████████| 127 kB 97.7 MB/s 
     |████████████████████████████████| 1.3 MB 32.5 MB/s 
     |████████████████████████████████| 5.5 MB 33.9 MB/s 
     |████████████████████████████████| 7.6 MB 61.4 MB/s 
     |████████████████████████████████| 352 kB 32.6 MB/s 


Imports

In [ ]:
import csv
import numpy as np
import pandas as pd
import torch
from bleurt import score
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, \
    MBart50TokenizerFast, MBartConfig,\
    TrainingArguments, Trainer
    

Mount google drive to use for file saving and loading

In [ ]:
from google.colab import files, drive
drive.flush_and_unmount()
drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive/
/content/gdrive/MyDrive


In [ ]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
ls: cannot open directory '.': Transport endpoint is not connected


Global variables

In [ ]:
ORIGINAL_MODEL_CHECKPOINT = 'facebook/mbart-large-50-many-to-many-mmt'
MODEL_CHECKPOINT = 'Mbart/Model/checkpoint-1000'
PAIRS = [
    'en-zh',
    'zh-en',
    'en-es',
    'es-zh',
    'es-en',
    'zh-es',
    ]
MBART_DATA = {
    'en-zh': {
        'size': 69020,
        'train': 48444,
        'val': 10381,
        'src': 'en',
        'tgt': 'zh',
        'src_tkn': 'en_XX',
        'tgt_tkn':'zh_CN',
        'tkn': 'zh_CN',
        'reverse': False,
        'train_path':f'Mbart/Data/en-zh-train_pairs.csv',
        'val_path':f'Mbart/Data/en-zh-val_pairs.csv',
        'test_path':f'Mbart/Data/en-zh-test_pairs.csv',
        },
    'zh-en': {
        'size': 69020,
        'train': 48444,
        'val': 10381,
        'src': 'zh',
        'tgt': 'en',
        'src_tkn': 'zh_CN',
        'tgt_tkn':'en_XX',
        'tkn': 'en_XX',
        'reverse': True,
        'train_path':f'Mbart/Data/en-zh-train_pairs.csv',
        'val_path':f'Mbart/Data/en-zh-val_pairs.csv',
        'test_path':f'Mbart/Data/en-zh-test_pairs.csv',
        },
    'en-es': {
        'size': 238511,
        'train': 167210,
        'val': 35831,
        'src': 'en',
        'tgt': 'es',
        'src_tkn': 'en_XX',
        'tgt_tkn':'es_XX',
        'tkn': 'es_XX',
        'reverse': False,
        'train_path':f'Mbart/Data/en-es-train_pairs.csv',
        'val_path':f'Mbart/Data/en-es-val_pairs.csv',
        'test_path':f'Mbart/Data/en-es-test_pairs.csv',
        },
    'es-zh': {
        'size': 65408,
        'train': 45796,
        'val': 9814,
        'src': 'es',
        'tgt': 'zh',
        'src_tkn': 'es_XX',
        'tgt_tkn':'zh_CN',
        'tkn': 'zh_CN',
        'reverse': False,
        'train_path':f'Mbart/Data/es-zh-train_pairs.csv',
        'val_path':f'Mbart/Data/es-zh-val_pairs.csv',
        'test_path':f'Mbart/Data/es-zh-test_pairs.csv',
        },
    'es-en': {
        'size': 238511,
        'train': 167210,
        'val': 35831,
        'src': 'es',
        'tgt': 'en',
        'src_tkn': 'es_XX',
        'tgt_tkn':'en_XX',
        'tkn': 'en_XX',
        'reverse': True,
        'train_path':f'Mbart/Data/en-es-train_pairs.csv',
        'val_path':f'Mbart/Data/en-es-val_pairs.csv',
        'test_path':f'Mbart/Data/en-es-test_pairs.csv',
        },
    'zh-es': {
        'size': 65408,
        'train': 45796,
        'val': 9814,
        'src': 'zh',
        'tgt': 'es',
        'src_tkn': 'zh_CN',
        'tgt_tkn':'es_XX',
        'tkn': 'es_XX',
        'reverse': True,
        'train_path':f'Mbart/Data/es-zh-train_pairs.csv',
        'val_path':f'Mbart/Data/es-zh-val_pairs.csv',
        'test_path':f'Mbart/Data/es-zh-test_pairs.csv',
        },
    }

DATASET = 'news_commentary'
MAX_LENGTH = 50
MAX_NEW_TOKENS = 50
TRUNCATION = True
PADDING = True
RETURN_TENSORS = 'pt'
BLEURT_CHECKPOINT = './BLEURT-20-D3'
N_EXAMPLES = 100

%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:256

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:256


In [ ]:
config = MBartConfig.from_pretrained(ORIGINAL_MODEL_CHECKPOINT)

config.encoder_layers = 1
config.decoder_layers = 1
config.num_hidden_layers = 1

Downloading:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Load Model, Metrics and Tokenizer

In [ ]:
# Load the metrics model and tokenizer for use in the next cell

bleurt_metric = score.LengthBatchingBleurtScorer(BLEURT_CHECKPOINT)
model = MBartForConditionalGeneration.from_pretrained(ORIGINAL_MODEL_CHECKPOINT, config=config)
tokenizer = MBart50TokenizerFast.from_pretrained(ORIGINAL_MODEL_CHECKPOINT)
# model = torch.load('Mbart/Model/Tiny/epoch-3')

Downloading:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mbart-large-50-many-to-many-mmt were not used when initializing MBartForConditionalGeneration: ['model.decoder.layers.1.encoder_attn.q_proj.weight', 'model.decoder.layers.9.encoder_attn.q_proj.weight', 'model.decoder.layers.11.final_layer_norm.weight', 'model.encoder.layers.9.fc1.bias', 'model.encoder.layers.4.fc1.weight', 'model.decoder.layers.7.fc1.bias', 'model.encoder.layers.6.self_attn.v_proj.bias', 'model.decoder.layers.1.self_attn.q_proj.weight', 'model.encoder.layers.8.fc2.weight', 'model.decoder.layers.7.self_attn.q_proj.bias', 'model.decoder.layers.5.encoder_attn.out_proj.bias', 'model.encoder.layers.2.final_layer_norm.weight', 'model.decoder.layers.9.encoder_attn.q_proj.bias', 'model.decoder.layers.5.final_layer_norm.bias', 'model.decoder.layers.6.final_layer_norm.weight', 'model.decoder.layers.5.encoder_attn.k_proj.weight', 'model.encoder.layers.3.self_attn.v_proj.bias', 'model.decoder.layers.6.self_attn_layer_norm.weight', '

Downloading:   0%|          | 0.00/529 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
model.config

MBartConfig {
  "_name_or_path": "Mbart/Model/Tiny/checkpoint-1000",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 3,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 3,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 200

In [ ]:

def eval_token(src_data, tokenizer, src_tkn):
  tokenizer.src_lang = src_tkn
 
  inputs = tokenizer(
        text=src_data,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=TRUNCATION,
        return_tensors=RETURN_TENSORS,
        )

  return {'input_ids':inputs['input_ids'].cuda(),
          'attention_mask':inputs['attention_mask'].cuda()}

In [ ]:
model.to('cuda')

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,

Scoring function definition

In [ ]:
# Function to generate dataset stream, select first n samples, generate predictions and return score and n samples used


def score_dataset(
    pair_index=0,
    starting_iter=0,
    end_iter=9999,
    n_examples=N_EXAMPLES,
    dataset=DATASET,
    model=model,
    tokenizer=tokenizer,
    bleurt_metric=bleurt_metric,
):
    """
    Evaluate the model that is passed into the function's performance on BLEU and
    BLEURT Translation performance. All results are output into csvs to store history
    external from the notebook.

    Inputs:
      Language Pair,
      Iteration Progress
      Ending Iteration,
      Number of examples
      Dataset
      Tokenizer
      Model,
      Bleurt Scorer

    Returns:
      Nothing

    """

    # Pair and properties for gatekeeping
    pair = PAIRS[pair_index]
    src = MBART_DATA[PAIRS[pair_index]]["src"]
    tgt = MBART_DATA[PAIRS[pair_index]]["tgt"]
    src_tkn = MBART_DATA[PAIRS[pair_index]]["src_tkn"]
    tgt_tkn = MBART_DATA[PAIRS[pair_index]]["tgt_tkn"]
    tkn = MBART_DATA[PAIRS[pair_index]]["tkn"]
    size = MBART_DATA[PAIRS[pair_index]]["size"]
    reverse = MBART_DATA[PAIRS[pair_index]]["reverse"]
    val = MBART_DATA[PAIRS[pair_index]]["val"]

    # Set Tokenizer up
    tokenizer.src_lang = src_tkn
    tokenizer.tgt_lang = tgt_tkn

    # Specify the scoring portion of the dataset
    eval = size * 0.01

    if eval > 10000:
      eval = 10000
    else:
      eval = 5000

    loops = int(eval // n_examples) - starting_iter + 1
    prev_iter = 0

    # Check where to start
    if starting_iter > 0:
        prev_iter = starting_iter

    # Check where to stop
    if end_iter == 9999:
        end_iter = loops
    else:
        loops = end_iter - starting_iter

    # Check if pair is reversed or not in the dataset loading
    if reverse:
        data_stream = load_dataset(DATASET, f"{tgt}-{src}", streaming=True)
    else:
        data_stream = load_dataset(DATASET, f"{src}-{tgt}", streaming=True)

    # Create/start your entries
    with open(f"Mbart/Single_bleurt.csv", "a", newline="") as bluert_file:
        wr = csv.writer(bluert_file, quoting=csv.QUOTE_ALL)
        wr.writerow([f"starting baseline scoring ------------- iter {prev_iter} ending at {end_iter}"])

    # Loop of scoring
    for run in range(loops + 1):
        start = prev_iter * n_examples

        # Normal loops of n_examples
        score_set = data_stream["train"].skip(start).take(n_examples)

        # Vectorize sentences
        src_data = [p["translation"][src] for p in score_set]
        tgt_data = [p["translation"][tgt] for p in score_set]

        # Progress text
        print(f"Now scoring iteration {run} of {loops}.")

        # Prepare inputs
        inputs = eval_token(src_data,tokenizer,src_tkn)

        # Pass inputs through the model
        generated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id[tkn],
            max_new_tokens=MAX_NEW_TOKENS,
        )
        gen = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        bleurt_score = bleurt_metric.score(references=tgt_data, candidates=gen)

        with open(f"Mbart/Single_bleurt.csv", "a", newline="") as bleurt_file:
            wr = csv.writer(bleurt_file, quoting=csv.QUOTE_ALL)
            wr.writerow([pair, run, np.mean(bleurt_score), "Baseline"])
        print(prev_iter+1, np.mean(bleurt_score))

        prev_iter += 1

    return None


Score the dataset

In [ ]:
# Load individual dataset, score BLEU and BLEURT scores for dataset
for p in range(0, len(PAIRS)):
    print(f"Evaluating translation {PAIRS[p]} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    score_dataset(
        pair_index=p,
        starting_iter=0,
        end_iter=9999,
        n_examples=N_EXAMPLES,
        dataset=DATASET,
        model=model,
        tokenizer=tokenizer,
        bleurt_metric=bleurt_metric,
    )


Evaluating translation en-zh ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Now scoring iteration 0 of 51.
1 0.06957114465534688
Now scoring iteration 1 of 51.
2 0.07684496589004994
Now scoring iteration 2 of 51.
3 0.07896293461322784
Now scoring iteration 3 of 51.
4 0.06644271694123745
Now scoring iteration 4 of 51.
5 0.07690199203789234
Now scoring iteration 5 of 51.
6 0.06937920421361923
Now scoring iteration 6 of 51.
7 0.06826732710003852
Now scoring iteration 7 of 51.
8 0.06737796001136304
Now scoring iteration 8 of 51.
9 0.06866659849882126
Now scoring iteration 9 of 51.
10 0.08107426285743713
Now scoring iteration 10 of 51.
11 0.07534064844250679
Now scoring iteration 11 of 51.
12 0.0653995593637228
Now scoring iteration 12 of 51.
13 0.07533326625823974
Now scoring iteration 13 of 51.
14 0.07171419404447078
Now scoring iteration 14 of 51.
15 0.06756175704300403
Now scoring iteration 15 of 51.
16 0.07119568981230259
Now scoring iteration 16 of 51.
17 0.08289370596408845
Now scoring iteration 17 of 51.
18 0.07135230667889118
Now scoring iteration 18 of 51

In [ ]:
!ls